<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 63.2 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=e740e4d813a2f85b4c95d69ba2b46cf070e9040dcf9e4d8208efbc7a30b7c09d
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta
 

In [3]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [25]:
ticker_yahoo = yf.Ticker('HINDZINC.NS')
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)


In [5]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [29]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

#print (ath_list)

def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ticker))
    return results

results = paralel_func(ath_list)


In [30]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)

# ticker_list.to_csv('Watchlist.csv', index=False)
ticker_list


,Ticker,Last_price,ATH,two_EMA
0,360ONE.NS,1838.800049,1960.000000,1701.202249
1,3MINDIA.NS,22995.449219,25209.000000,22206.125703
2,ABBOTINDIA.NS,20978.949219,22499.000000,19260.241035
3,AEGISCHEM.NS,362.049988,388.799988,277.581000
4,AGARIND.NS,686.150024,746.200012,594.227999
5,AKZOINDIA.NS,2222.500000,2400.000000,2044.508752
6,ANANDRATHI.NS,802.599976,862.500000,681.024500
7,ARMANFIN.NS,1610.199951,1705.000000,1326.531252
8,ASTRAZEN.NS,3271.649902,3620.000000,3100.373000
9,ATFL.NS,882.299988,969.799988,808.872999


In [224]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today() + relativedelta(days=1)

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df = yf.download(tickers, start= startdate, end= enddate)
df

[*********************100%***********************]  119 of 119 completed


Adj Close                                                       \
              360ONE.NS    3MINDIA.NS ABBOTINDIA.NS AEGISCHEM.NS  AGARIND.NS   
Date                                                                           
2019-02-08          NaN  20586.744141   7712.213867   182.931732  155.136246   
2019-02-11          NaN  20863.470703   7352.016113   182.266342  150.136490   
2019-02-12          NaN  20472.099609   7436.551270   183.502060  151.434509   
2019-02-14          NaN  20587.468750   7331.254395   181.173248  154.655502   
2019-02-15          NaN  20177.953125   7137.250488   167.818115  161.482071   
2019-02-18          NaN  21467.687500   6966.310547   171.097488  151.867172   
2019-02-19          NaN  21161.044922   6938.067871   168.245850  158.164948   
2019-02-20          NaN  21203.843750   6995.942871   176.325470  156.722702   
2019-02-21          NaN  21941.328125   6949.048828   182.836655  155.857361   
2019-02-22          NaN  22545.832031   6934.039551   187.684448  157.636124   
2019-02-25          NaN  22821.689453   7072.566406   189.585480  161.626312   
2019-02-26          NaN  22099.451172   7057.989258   190.345917  169.029770   
2019-02-27          NaN  22023.935547   7070.360352   185.973450  163.164688   
2019-02-28          NaN  22095.880859   7132.599121   189.062714  162.635864   
2019-03-01          NaN  22207.101562   7032.672363   196.857178  163.789658   
2019-03-05          NaN  22630.902344   6998.100098   196.524475  162.635864   
2019-03-06          NaN  22439.291016   7060.770996   194.908554  164.462692   
2019-03-07          NaN  23001.333984   6982.085449   192.389603  164.606918   
2019-03-08          NaN  23979.550781   7061.873535   195.906631  166.241425   
2019-03-11          NaN  23682.558594   7093.711914   195.764038  163.645416   
2019-03-12          NaN  23659.589844   7053.818848   196.572006  162.395477   
2019-03-13          NaN  23357.531250   7051.612305   192.104462  163.453110   
2019-03-14          NaN  23095.666016   7030.370605   194.480789  160.760956   
2019-03-15          NaN  22912.115234   6992.490234   193.862930  157.828400   
2019-03-18          NaN  22439.050781   7060.386230   194.813492  162.058960   
2019-03-19          NaN  22695.173828   7057.365723   193.910461  159.030273   
2019-03-20          NaN  22620.769531   7054.248535   190.155838  159.655243   
2019-03-22          NaN  22760.554688   7081.053223   191.771759  155.136246   
2019-03-25          NaN  23307.203125   7096.348633   191.914337  151.386429   
2019-03-26          NaN  23112.312500   7045.187500   190.961441  151.626801   
2019-03-27          NaN  22946.664062   6930.971680   193.963089  147.876999   
2019-03-28          NaN  23195.451172   6976.571289   191.247314  157.347656   
2019-04-01          NaN  23445.494141   6997.621094   196.917053  156.818848   
2019-04-02          NaN  23454.421875   7009.464844   195.582993  154.799728   
2019-04-03          NaN  23145.125000   7028.452637   195.535370  154.078598   
2019-04-04          NaN  22828.011719   6962.666016   194.296600  153.790161   
2019-04-05          NaN  23027.535156   7002.751953   205.302597  153.645935   
2019-04-08          NaN  23042.734375   7015.170410   196.297684  151.915253   
2019-04-09          NaN  22542.550781   7031.617676   201.776871  149.944199   
2019-04-10          NaN  23445.060547   7037.322754   199.966354  150.857620   
2019-04-11          NaN  23731.001953   7065.565430   198.918152  147.059723   
2019-04-12          NaN  23924.929688   7096.828613   198.584656  147.973145   
2019-04-15          NaN  24124.693359   7506.126465   201.967468  150.521103   
2019-04-16          NaN  24053.085938   7701.185547   200.919281  145.280991   
2019-04-18          NaN  24106.017578   7504.208984   198.870529  172.202682   
2019-04-22          NaN  24157.503906   7398.670898   195.535370  171.385406   
2019-04-23          NaN  24278.423828   7470.404297   180.098389  170.472000   
2019-04-24    

In [225]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

df1 = pd.DataFrame()


for ticker in tickers:
  test = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
  test1 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
  test2 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
  test3 = ta.ema(close=ax_df[('High', ticker)], length=32).to_frame('EMA200_H')
  test4 = ta.ema(close=ax_df[('Close', ticker)], length=32).to_frame('EMA200_C')
  test5 = ta.ema(close=ax_df[('Low', ticker)], length=32).to_frame('EMA200_L')
  test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
  test1.columns = pd.MultiIndex.from_product([test1.columns, [ticker]])
  test2.columns = pd.MultiIndex.from_product([test2.columns, [ticker]])
  test3.columns = pd.MultiIndex.from_product([test3.columns, [ticker]])
  test4.columns = pd.MultiIndex.from_product([test4.columns, [ticker]])
  test5.columns = pd.MultiIndex.from_product([test5.columns, [ticker]])
  df1 = pd.concat([df1, test, test1, test2, test3, test4, test5], axis=1)
  df1.sort_index()
  
ax_df = pd.concat([ax_df, df1], axis=1)
ax_df.reset_index()
ax_df = ax_df.drop(['Adj Close','Low','High','Volume'], level=0, axis=1)


In [226]:
ax_df= ax_df.swaplevel(0, 1, 1).sort_index(axis=1)

In [227]:
ax_df= ax_df.tail(2)

In [229]:
from time import strftime
import datetime
prev_day = (date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
to_day = date.today().strftime('%Y-%m-%d')


In [231]:
to_day

'2023-02-08'

In [232]:
df2= pd.DataFrame()
for ticker in tickers:
  cond1 = (ax_df.loc[to_day:,(ticker,'Close')] > ax_df.loc[to_day:,(ticker,'EMA200_H')]).bool()
  cond2 = (ax_df.loc[to_day:,(ticker,'Close')] > ax_df.loc[to_day:,(ticker,'Open')]).bool()
  cond3 = (ax_df.loc[prev_day:,(ticker,'SUPERTd_12_3.0')].to_numpy() < ax_df.loc[to_day:,(ticker,'SUPERTd_12_3.0')].to_numpy())
  #cond3 = (ax_df.loc[:,(ticker,'SUPERTd_10_1.0')] + ax_df.loc[:,(ticker,'SUPERTd_11_2.0')] + ax_df.loc[:,(ticker,'SUPERTd_12_3.0')] > -1).bool()

  if cond1 and cond2 and cond3[0]:
    ax_df1 = ax_df.loc[:,(ticker)]
    ax_df1.columns = pd.MultiIndex.from_product([[ticker],ax_df1.columns])
    df2 = pd.concat([df2,ax_df1], axis=1)
    
#df2.to_csv('test.csv')


In [233]:
pd.set_option('display.max_rows', None)
df2 = df2.T
df2

Date                           2023-02-07   2023-02-08
DYNAMATECH.NS Close           2541.300049  2657.550049
              EMA200_C        2461.171908  2473.073613
              EMA200_H        2510.498529  2526.044073
              EMA200_L        2414.315326  2421.629549
              Open            2474.949951  2535.000000
              SUPERT_10_1.0   2393.477757  2535.529981
              SUPERT_11_2.0   2562.644403  2424.762981
              SUPERT_12_3.0   2643.685051  2317.523246
              SUPERTd_10_1.0     1.000000     1.000000
              SUPERTd_11_2.0    -1.000000     1.000000
              SUPERTd_12_3.0    -1.000000     1.000000
              SUPERTl_10_1.0  2393.477757  2535.529981
              SUPERTl_11_2.0          NaN  2424.762981
              SUPERTl_12_3.0          NaN  2317.523246
              SUPERTs_10_1.0          NaN          NaN
              SUPERTs_11_2.0  2562.644403          NaN
              SUPERTs_12_3.0  2643.685051          NaN
HITECHGEAR.NS Close            267.200012   273.200012
              EMA200_C         256.699175   257.699226
              EMA200_H         264.240880   264.905070
              EMA200_L         251.034820   251.350892
              Open             255.000000   262.500000
              SUPERT_10_1.0    252.047998   252.047998
              SUPERT_11_2.0    238.281615   238.281615
              SUPERT_12_3.0    271.589041   223.832806
              SUPERTd_10_1.0     1.000000     1.000000
              SUPERTd_11_2.0     1.000000     1.000000
              SUPERTd_12_3.0    -1.000000     1.000000
              SUPERTl_10_1.0   252.047998   252.047998
              SUPERTl_11_2.0   238.281615   238.281615
              SUPERTl_12_3.0          NaN   223.832806
              SUPERTs_10_1.0          NaN          NaN
              SUPERTs_11_2.0          NaN          NaN
              SUPERTs_12_3.0   271.589041          NaN